# LLaVA: Large Language and Vision Assistant

![LLAVA.png](https://velog.velcdn.com/images/tm011899/post/e81c2c49-0dbb-42be-8a3f-f39c436a87e1/image.png)

H Liu, C Li, Q Wu, YJ Lee. arXiv:2304.08485, 2023

https://arxiv.org/abs/2304.08485

https://llava-vl.github.io/

LLaVA (Large Language and Vision Assistant) 모델은 언어만을 사용하는 GPT-4를 활용하여 다중 모달 언어-이미지 지시 데이터를 생성하는 새로운 접근 방식을 제시합니다. 이 논문에서는 시각 인코더(CLIP ViT-L/14)와 대규모 언어 모델(LLaMA기반 Vicuna)을 연결하여 일반적인 시각 및 언어 이해를 위한 대규모 멀티모달 모델을 개발하는 것을 목표로 합니다

## Instroduction


1. **멀티모달 insturction-following 데이터**

    주요한 요소중 하나는 vision-language instruction-following 데이터의 부족이다. 이 논문에서 단순히 image와 text를 연결시킨 기존의 image-text모델 데이터 구축방법에서 관점을 바꿔 ChatGPT/GPT-4를 사용해서 직접 이미지에 대한 정보를 LLM에 입력하고 출력하여 image-text 쌍을 instruction-following 형식으로 변환시켰다.

2. **Large Multimodal Model**

    이 논문에서는 CLIP ViT-L/14의 visual encoder와 LLaMA의 language encoder를 연결시켜서 large multimodal model(LMM)을 구성한다.

3. **Open-source**

    논문을 기반으로 생성된 멀티모달 명령 데이터, 데이터 생성 및 모델 훈련을 위한 코드베이스, 모델 체크포인트, 시각적 챗봇 데모 등을 공개하였다.

## LLAVA 모델의 훈련 데이터 준비 과정

데이터 선별: 연구팀은 Conceptual Captions 3M (CC3M) 데이터셋에서 595K개의 이미지-텍스트 쌍을 필터링했습니다.

![image](https://github.com/Mutoy-choi/Study_Deeplearning/assets/87027571/b28b47c1-29d5-4709-99e0-26821c176ab4)

데이터 변환: 선별된 이미지-텍스트 쌍은 '단순 확장 방법'을 사용하여 명령-따르기 형식의 데이터로 변환되었습니다. 기존에 멀티모달 분야에서 사용하던 사람이 직접 annotation하여 이러한 형식의 대규모 데이터를 만드는 방식은 매우 비싼 cost가 필요합니다. 하지만 이 방법은 ChatGPT/GPT-4를 활용하여 각 이미지에 대한 설명을 얻고, 이를 위의 Table 1에 제시된 것 처럼 언어 모델에 입력하여 생성된 텍스트를 기반으로 새로운 데이터 쌍을 형성하여 더 낮은 cost로 사람이 직접 작업한것 보다 좋은 품질의 데이터를 생성합니다.

여기서의 특징점은 instruction-following data를 만드는 방식인데, 특정 prompt를 ChatGPT/GPT-4의 input으로 사용합니다. 이때 주의해야할 것은 이미지는 input으로 사용하지 않고, 이미지와 관련된 캡션 및 Bounding box 값들만 이용해 질문 및 대화 셋을 만듭니다. 그후 ChatGPT를 이용해 질문하고 답하는 대화 형식(type1: conversation)의 데이터를 생성합니다. 이와 유사하지만 다른 prompt를 사용해, 더 자세한 설명(type2: detailed description) 혹은 복잡한 추론(type3: complex reasoning) 데이터를 생성할 수 있습니다.(table 1 참조)

데이터 형식: 변환된 데이터는 단일 턴 대화 형태로 처리되어, 모델이 이미지에 대한 정보를 이해하고 대응하는 방식을 학습할 수 있도록 합니다.

이미지에 대한 설명을 텍스트 데이터로 ChatGPT/GPT-4에 입력해주어 단순히 이미지를 넣어서 데이터 쌍을 형성한 것에 비해 실제 세계의 시각적 상황을 언어적으로 해석하고 반응할 수 있는 능력을 개발하는 데 필요한 다양하고 복잡한 데이터를 확보할 수 있었습니다.


## LLaVA 모델의 구조와 학습 과정

LLaVA모델의 구조

![image](https://llava-vl.github.io/images/llava_arch.png)

LLaVA 모델의 학습 과정은 다음과 같이 진행되었습니다:

1. **사전 훈련 (Pre-training):**
연구팀은 이미지와 텍스트 쌍 595K개를 '단순 확장 방법'을 사용하여 명령-따르기 데이터로 변환했습니다. 이 데이터는 각 샘플이 단일 턴 대화로 처리되도록 구성되었습니다. 이 단계에서는 이미지와 텍스트 데이터를 결합하여 모델이 시각적 정보를 언어적으로 해석하고 반응하는 방식을 학습하게 됩니다

2. **종단간 정밀 훈련 (Fine-tuning End-to-End):**
사전 훈련 후, 연구팀은 시각 인코더 가중치를 고정하고, 프로젝션 레이어와 LLaMA의 사전 훈련된 가중치를 계속 업데이트했습니다. 이 단계에서는 멀티턴과 싱글턴 대화가 가능한 챗봇을 개발하기 위해 158K개의 고유한 언어-이미지 명령-따르기 데이터를 사용했습니다

3. **학습 목표 설정:**
모델의 주요 학습 목표는 이미지 정보와 관련된 언어적 질문에 대해 효과적으로 대응하는 것입니다. 이를 위해 다양한 유형의 질문과 시각적 정보를 결합하여 모델의 이해력과 반응력을 강화하는 데 중점을 두었습니다.

4. **성능 평가:**
학습 과정 동안 모델의 성능은 정량적 평가를 통해 지속적으로 검증되었습니다. 이는 GPT-4를 활용하여 모델의 생성 응답의 질을 측정하는 방식으로 진행되었습니다.


이러한 학습 과정을 통해 LLAVA 모델은 다양한 시각적 및 언어적 작업을 수행하는 데 필요한 능력을 키우고, 실세계의 복잡한 시나리오에 대해 효과적으로 대응할 수 있게 되었습니다.